In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image

from utils_new import mnist, plot_graphs, plot_mnist
import numpy as np
import os 

%matplotlib inline

In [2]:
def to_onehot(x, n, device=None):
    if isinstance(x, np.ndarray):
        x = torch.Tensor(x).to(torch.long)
    one_hot = torch.zeros((x.shape[0], n))
    one_hot.scatter_(1, x[:, None], 1.)
    if device is not None:
        one_hot = one_hot.to(device)
    return one_hot

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
root_folder = 'FC_GAE_results'
fixed_folder = root_folder + '/Fixed_results/'
recon_folder = root_folder + '/Recon_results/'

if os.path.isdir(root_folder):
    !rm -r $root_folder
os.mkdir(root_folder)
os.mkdir(fixed_folder)
os.mkdir(recon_folder)

In [5]:
mnist_tanh = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),
                lambda x: x.to(device)
           ])

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

lr = 0.0001
prior_size = 10
train_epoch = 1000
batch_size = 250
train_loader, valid_loader, test_loader = mnist(batch_size=batch_size, valid=10000, transform=mnist_tanh)
fixed_z = torch.randn((10, prior_size)).repeat((1,10)).view(-1, prior_size).to(device)
fixed_z_label = to_onehot(torch.tensor(list(range(10))).repeat((10)), 10).to(device)
fixed_data, fixed_label = next(iter(test_loader))
fixed_data = fixed_data[:100].to(device)
fixed_label = to_onehot(fixed_label[:100], 10).to(device)

cpu


In [7]:
data, label = next(iter(train_loader))

In [8]:
class FullyConnected(nn.Module):
    def __init__(self, sizes, dropout=False, activation_fn=nn.Tanh(), flatten=False, 
                 last_fn=None, first_fn=None, device='cpu'):
        super(FullyConnected, self).__init__()
        layers = []
        self.flatten = flatten
        if first_fn is not None:
            layers.append(first_fn)
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if dropout:
                layers.append(nn.Dropout(dropout))
            layers.append(activation_fn) # нам не нужен дропаут и фнкция активации в последнем слое
        else: 
            layers.append(nn.Linear(sizes[-2], sizes[-1]))
        if last_fn is not None:
            layers.append(last_fn)
        self.model = nn.Sequential(*layers)
        self.to(device)
        
    def forward(self, x, y=None):
        if self.flatten:
            x = x.view(x.shape[0], -1)
        if y is not None:
            x = torch.cat([x, y], dim=1)
        return self.model(x)

In [9]:
Enc = FullyConnected([28*28, 1024, 1024, prior_size], activation_fn=nn.LeakyReLU(0.2), flatten=True, device=device)

Dec = FullyConnected([prior_size, 1024, 1024, 28*28], activation_fn=nn.LeakyReLU(0.2), last_fn=nn.Tanh(), device=device)
Disc = FullyConnected([prior_size, 1024, 1024, 1], dropout=0.3, activation_fn=nn.LeakyReLU(0.2), device=device)

Enc_optimizer = optim.Adam(Enc.parameters(), lr=lr)
Dec_optimizer = optim.Adam(Dec.parameters(), lr=lr)
Disc_optimizer = optim.Adam(Disc.parameters(), lr=lr)

In [10]:
train_log = {'E': [],'AE': [], 'D': []}
test_log = {'E': [],'AE': [], 'D': []}

In [11]:
batch_zeros = torch.zeros((batch_size, 1)).to(device)
batch_ones = torch.ones((batch_size, 1)).to(device)

In [12]:
def train(epoch, Enc, Dec, Disc, log=None):
    train_size = len(train_loader.sampler)
    for batch_idx, (data, label) in enumerate(train_loader):
        label = to_onehot(label, 10, device)
        # train D
        Enc.zero_grad()
        Disc.zero_grad()
        
        z = torch.randn((batch_size, prior_size)).to(device)

        fake_pred = Disc(Enc(data))
        true_pred = Disc(z)

        
        fake_loss = F.binary_cross_entropy_with_logits(fake_pred, batch_zeros)
        true_loss = F.binary_cross_entropy_with_logits(true_pred, batch_ones)
        
        Disc_loss = 0.5*(fake_loss + true_loss)
        
        Disc_loss.backward()
        Disc_optimizer.step()
        # train AE
        Enc.zero_grad()
        Dec.zero_grad()
        Disc.zero_grad()
        
        z = torch.randn((batch_size, prior_size))
        
        latent = Enc(data)
        
        reconstructed = Dec(latent).view(-1, 1, 28, 28)
        fake_pred = Disc(latent)
        
        Enc_loss = F.binary_cross_entropy_with_logits(fake_pred, batch_ones)
        AE_loss = F.mse_loss(reconstructed, data)
        G_loss = AE_loss + Enc_loss
        
        G_loss.backward()
        Dec_optimizer.step()
        Enc_optimizer.step()
            
        if batch_idx % 100 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
            losses = 'E: {:.4f}, AE: {:.4f}, D: {:.4f}'.format(Enc_loss.item(), AE_loss.item(), Disc_loss.item())
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
        losses = 'E: {:.4f}, AE: {:.4f}, D: {:.4f}'.format(Enc_loss.item(), AE_loss.item(), Disc_loss.item())
        print(line + losses)
        log['E'].append(Enc_loss.item())
        log['AE'].append(AE_loss.item())
        log['D'].append(Disc_loss.item())

In [13]:
def test(Enc, Dec, Disc, loader, epoch, log=None):
    test_size = len(loader)
    E_loss = 0.
    AE_loss = 0.
    D_loss = 0.
    test_loss = {'E': 0., 'AE': 0., 'D': 0.}
    with torch.no_grad():
        for data, label in loader:
            label = to_onehot(label, 10, device)
            z = torch.randn((batch_size, prior_size)).to(device)
            latent = Enc(data)
            reconstructed = Dec(latent).view(-1, 1, 28, 28)
            fake_pred = Disc(latent)
            true_pred = Disc(z)
        
            fake_loss = F.binary_cross_entropy_with_logits(fake_pred, batch_zeros).item()
            true_loss = F.binary_cross_entropy_with_logits(true_pred, batch_ones).item()
            
            D_loss += 0.5*(fake_loss + true_loss)
            E_loss += F.binary_cross_entropy_with_logits(fake_pred, batch_ones).item()
            AE_loss += F.mse_loss(reconstructed, data)
            
        E_loss /= test_size
        D_loss /= test_size
        AE_loss /= test_size

        fixed_gen = Dec(fixed_z).cpu().data.numpy().reshape(100, 1, 28, 28)
        plot_mnist(fixed_gen, (10, 10), False, fixed_folder + '%03d.png' % epoch)
        fixed_reconstruction = Dec(Enc(fixed_data)).cpu().data.numpy().reshape(100, 1, 28, 28)
        plot_mnist(fixed_reconstruction, (10, 10), False, recon_folder + '%03d.png' % epoch)
        
    report = 'Test losses. E: {:.4f}, AE: {:.4f}, D: {:.4f}'.format(E_loss, AE_loss, D_loss)
    print(report)

In [ ]:
for epoch in range(1, 1001):
    Enc.train()
    Dec.train()
    Disc.train()
    train(epoch, Enc, Dec, Disc, train_log)
    Enc.eval()
    Dec.eval()
    Disc.eval()
    test(Enc, Dec, Disc, valid_loader, epoch, test_log)
    

Train Epoch: 1 [0/50000 (0%)]	Losses E: 0.6809, AE: 0.9240, D: 0.6987
Train Epoch: 1 [25000/50000 (50%)]	Losses E: 1.4953, AE: 0.2748, D: 5.7161
Train Epoch: 1 [50000/50000 (100%)]	Losses E: 0.8699, AE: 0.2786, D: 1.6772
Test losses. E: 0.8521, AE: 0.2751, D: 1.6651
Train Epoch: 2 [0/50000 (0%)]	Losses E: 0.8569, AE: 0.2762, D: 1.6244
Train Epoch: 2 [25000/50000 (50%)]	Losses E: 0.6200, AE: 0.2628, D: 1.0018
Train Epoch: 2 [50000/50000 (100%)]	Losses E: 0.9810, AE: 0.2660, D: 0.8875
Test losses. E: 0.6697, AE: 0.2675, D: 0.9060
Train Epoch: 3 [0/50000 (0%)]	Losses E: 1.0087, AE: 0.2601, D: 0.8616
Train Epoch: 3 [25000/50000 (50%)]	Losses E: 0.5828, AE: 0.2662, D: 0.9127
Train Epoch: 3 [50000/50000 (100%)]	Losses E: 0.4959, AE: 0.2499, D: 0.8243
Test losses. E: 0.4575, AE: 0.2564, D: 0.8212
Train Epoch: 4 [0/50000 (0%)]	Losses E: 0.4882, AE: 0.2549, D: 0.8317
Train Epoch: 4 [25000/50000 (50%)]	Losses E: 0.7889, AE: 0.2428, D: 0.8168
Train Epoch: 4 [50000/50000 (100%)]	Losses E: 0.5526, 

Train Epoch: 31 [50000/50000 (100%)]	Losses E: 0.6824, AE: 0.1181, D: 0.6978
Test losses. E: 0.6817, AE: 0.1169, D: 0.6974
Train Epoch: 32 [0/50000 (0%)]	Losses E: 0.6862, AE: 0.1181, D: 0.6945
Train Epoch: 32 [25000/50000 (50%)]	Losses E: 0.6481, AE: 0.1184, D: 0.7043
Train Epoch: 32 [50000/50000 (100%)]	Losses E: 0.6445, AE: 0.1182, D: 0.7252
Test losses. E: 0.6385, AE: 0.1223, D: 0.7255
Train Epoch: 33 [0/50000 (0%)]	Losses E: 0.6413, AE: 0.1229, D: 0.7265
Train Epoch: 33 [25000/50000 (50%)]	Losses E: 0.7477, AE: 0.1277, D: 0.7159
Train Epoch: 33 [50000/50000 (100%)]	Losses E: 0.6299, AE: 0.1109, D: 0.7038
Test losses. E: 0.6245, AE: 0.1122, D: 0.7058
Train Epoch: 34 [0/50000 (0%)]	Losses E: 0.6271, AE: 0.1141, D: 0.7057
Train Epoch: 34 [25000/50000 (50%)]	Losses E: 0.8236, AE: 0.1453, D: 0.7101
Train Epoch: 34 [50000/50000 (100%)]	Losses E: 0.7229, AE: 0.1198, D: 0.6884
Test losses. E: 0.7167, AE: 0.1157, D: 0.6883
Train Epoch: 35 [0/50000 (0%)]	Losses E: 0.7200, AE: 0.1110, D: 0.6

Train Epoch: 62 [0/50000 (0%)]	Losses E: 0.6373, AE: 0.0835, D: 0.6814
Train Epoch: 62 [25000/50000 (50%)]	Losses E: 0.6170, AE: 0.1137, D: 0.7293
Train Epoch: 62 [50000/50000 (100%)]	Losses E: 0.6715, AE: 0.0948, D: 0.6989
Test losses. E: 0.6716, AE: 0.0954, D: 0.6976
Train Epoch: 63 [0/50000 (0%)]	Losses E: 0.6725, AE: 0.0959, D: 0.6981
Train Epoch: 63 [25000/50000 (50%)]	Losses E: 0.6507, AE: 0.0976, D: 0.7043
Train Epoch: 63 [50000/50000 (100%)]	Losses E: 0.7024, AE: 0.0890, D: 0.6844
Test losses. E: 0.6981, AE: 0.0939, D: 0.6856
Train Epoch: 64 [0/50000 (0%)]	Losses E: 0.7022, AE: 0.0916, D: 0.6844
Train Epoch: 64 [25000/50000 (50%)]	Losses E: 0.7429, AE: 0.0966, D: 0.7052
Train Epoch: 64 [50000/50000 (100%)]	Losses E: 0.7153, AE: 0.0895, D: 0.6978
Test losses. E: 0.7105, AE: 0.0900, D: 0.6982
Train Epoch: 65 [0/50000 (0%)]	Losses E: 0.7121, AE: 0.0873, D: 0.6987
Train Epoch: 65 [25000/50000 (50%)]	Losses E: 0.7047, AE: 0.0817, D: 0.6921
Train Epoch: 65 [50000/50000 (100%)]	Losses